In [ ]:
#Develop a Python program that detects red-eye in images

In [ ]:
#pip install opencv-python

In [ ]:
# Red-eye is a common phenomenon in flash photography where the pupils of human eyes appear red in the captured images.
#Develop a Python program that detects red-eye in images. 

In [ ]:
import cv2
import numpy as np

def custom_red_eye_detection(image_path):
    # Load the image
    image = cv2.imread(image_path)
    
    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Load the Haarcascades for face and eye detection
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

    # Detect faces in the image
    detected_faces = face_cascade.detectMultiScale(gray_image, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    # Define a range of red color in HSV
    lower_red = np.array([0, 100, 100])
    upper_red = np.array([10, 255, 255])

    # Loop through detected faces
    for (fx, fy, fw, fh) in detected_faces:
        face_region = image[fy:fy+fh, fx:fx+fw]
        gray_face = gray_image[fy:fy+fh, fx:fx+fw]
        
        # Detect eyes within the face region
        detected_eyes = eye_cascade.detectMultiScale(gray_face)
        
        # Loop through detected eyes
        for (ex, ey, ew, eh) in detected_eyes:
            eye_region = face_region[ey:ey+eh, ex:ex+ew]
            hsv_eye = cv2.cvtColor(eye_region, cv2.COLOR_BGR2HSV)
            
            # Create a mask for red color
            red_mask = cv2.inRange(hsv_eye, lower_red, upper_red)
            
            # Calculate the percentage of red pixels
            total_pixels = red_mask.size
            red_pixel_count = np.count_nonzero(red_mask)
            red_pixel_percentage = (red_pixel_count / total_pixels) * 100
            
            # If the red pixel percentage is above a threshold, consider it as red-eye
            if red_pixel_percentage > 2:
                # Draw a rectangle around the detected red eye
                cv2.rectangle(face_region, (ex, ey), (ex + ew, ey + eh), (0, 0, 255), 2)
        
        # Copy the face region with red-eye rectangles back to the original image
        image[fy:fy+fh, fx:fx+fw] = face_region

    # Display the image with detected red eyes
    cv2.imshow('Custom Red Eye Detection', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Specify the path to the image
image_path = 'C:/Users/Admin/Downloads/gagarin_2.jpg'

# Call the custom function for red eye detection
custom_red_eye_detection(image_path)


In [ ]:
import cv2
import numpy as np
import requests

def detect_red_eye(image):
    # Convert the image to the LAB color space
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2Lab)

    # Split the LAB channels
    l_channel, a_channel, b_channel = cv2.split(lab_image)

    # Apply thresholding on the 'a' channel to detect potential red-eye regions
    _, a_thresh = cv2.threshold(a_channel, 150, 255, cv2.THRESH_BINARY)

    # Perform morphological operations to enhance the regions
    kernel = np.ones((3, 3), np.uint8)
    a_thresh = cv2.morphologyEx(a_thresh, cv2.MORPH_CLOSE, kernel, iterations=2)
    a_thresh = cv2.morphologyEx(a_thresh, cv2.MORPH_OPEN, kernel, iterations=2)

    # Find contours in the thresholded image
    contours, _ = cv2.findContours(a_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Draw rectangles around detected red-eye regions
    for contour in contours:
        area = cv2.contourArea(contour)
        if area < 70 :  # Filter out small regions
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 0, 255), 2)

    return image

# Load the image

image_url= "https://tse1.mm.bing.net/th?id=OIP.sGr2RyXltZGebfNUyC9jGgHaHr&pid=Api&P=0&h=180"

# Fetch the image from the URL
response = requests.get(image_url)
image_data = response.content

# Convert the image data into a numpy array
image_array = np.frombuffer(image_data, np.uint8)
image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)

# Call the function to detect eyes using color segmentation
result_image = detect_red_eye(image)

# Display the original and result images
cv2.imshow("Original Image", image)
cv2.imshow("Image with Detected Eyes", result_image)
cv2.waitKey(10000)
cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np
import requests

def detect_red_eye(image):
    # Convert the image to the LAB color space
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2Lab)

    # Split the LAB channels
    l_channel, a_channel, b_channel = cv2.split(lab_image)

    # Apply thresholding on the 'a' channel to detect potential red-eye regions
    _, a_thresh = cv2.threshold(a_channel, 150, 255, cv2.THRESH_BINARY)

    # Perform morphological operations to enhance the regions
    kernel = np.ones((3, 3), np.uint8)
    a_thresh = cv2.morphologyEx(a_thresh, cv2.MORPH_CLOSE, kernel, iterations=2)
    a_thresh = cv2.morphologyEx(a_thresh, cv2.MORPH_OPEN, kernel, iterations=2)

    # Find contours in the thresholded image
    contours, _ = cv2.findContours(a_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Sort the contours by area in descending order
    contours = sorted(contours, key=cv2.contourArea, reverse=True)

    # Draw rectangles around the two largest detected regions (eyes)
    for contour in contours[:2]:
        area = cv2.contourArea(contour)
        if area > 70:  # Filter out small regions
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 0, 255), 2)

    return image

# Load the image

image_url = "https://tse1.mm.bing.net/th?id=OIP.sGr2RyXltZGebfNUyC9jGgHaHr&pid=Api&P=0&h=180"

# Fetch the image from the URL
response = requests.get(image_url)
image_data = response.content

# Convert the image data into a numpy array
image_array = np.frombuffer(image_data, np.uint8)
image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)

# Call the function to detect eyes using color segmentation
result_image = detect_red_eye(image)

# Display the original and result images
cv2.imshow("Original Image", image)
cv2.imshow("Image with Detected Eyes", result_image)
cv2.waitKey(10000)
cv2.destroyAllWindows()
